<a href="https://colab.research.google.com/github/esheshka/team_one_project/blob/main/%D0%A7%D1%82%D0%BE_%D1%82%D0%BE_%D0%B7%D0%B0%D1%87%D0%B5%D0%BC_%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [4]:
from google.colab import drive; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 26jun23                  # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

- competition is now set to: 26jun23
Using competition: 26jun23
  teamId  teamName                   submissionDate       score    
--------  -------------------------  -------------------  -------  
10612695  Ne1tr0N                    2023-06-27 15:03:47  0.86489  
10605413  💳baseline,LR:$Stat+MCCEnc  2023-06-25 18:15:10  0.53220  


In [6]:
%%time
%%capture
%reset -f
!pip install -U sentence-transformers tfds-nightly tensorflow tensorflow_addons plotly > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"  # allows multiple prints from a cell
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly.express as px, xgboost as xgb, lightgbm, re, tensorflow as tf, tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier   # SKLearn's MLP is optimised for CPU (and doesn't use GPU)
from sentence_transformers import SentenceTransformer as SBERT
from keras.layers import Flatten, Dense

os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'   # helps to seed TF's RNG on GPUs
np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 10, 'display.width', 1000, 'display.max_rows', 8)

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*2): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

CPU times: user 9.79 s, sys: 1.51 s, total: 11.3 s
Wall time: 1min 27s


In [8]:
#@title MCC
%time mcc = pd.read_csv('_mcc.csv', index_col='mcc'); mcc  # load merchant category codes (MCC)

CPU times: user 14.3 ms, sys: 0 ns, total: 14.3 ms
Wall time: 42.8 ms


,desc
mcc,
0,Ветеринарные услуги
1,"Генеральные подрядчики по вентиляции, теплоснабжению, и водопроводу"
2,Подрядчики по электричеству
3,"Подрядчики, специализированная торговля — нигде более не классифицированные"
...,...
180,Штрафы
181,Налоговые платежи
182,"Правительственные услуги, нигде ранее не классифицируемые"
183,Почтовые услуги — только правительство


In [12]:
#@title TTC
%time ttc = pd.read_csv('_ttc.csv', index_col='ttc'); ttc  # load transaction type codes (TTC)

CPU times: user 4.53 ms, sys: 0 ns, total: 4.53 ms
Wall time: 6.46 ms


,desc
ttc,
0,Покупка.ТУ СБ РФ
1,Покупка. POS ТУ СБ РФ
2,Оплата услуги. Банкоматы СБ РФ
3,Покупка. ТУ Россия
...,...
151,Плата за приост. действия карты
152,Плата за обработку счета банковской карты клиента при осущ. платежной системой экстренной выдачи...
153,Плата за предоставление по запросу клиента документов по операции с картой
154,XXX


In [145]:
#@title Gender
%time gender = pd.read_csv('gender.csv', index_col='cid'); gender.T  # load gender labels. you need to predict the missing labels
trainY, testY = gender.query('gender==gender'), gender.query('gender!=gender')  # split into Train and test (=Validation) targets

CPU times: user 7.51 ms, sys: 0 ns, total: 7.51 ms
Wall time: 12 ms


cid,0,1,2,3,4,5,6,7,8,9,...,8390,8391,8392,8393,8394,8395,8396,8397,8398,8399
gender,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [13]:
#@title TRX
%time dfTrx = pd.read_csv('trx.csv').sort_values(['cid','dt','tm']); dfTrx  # load bank card's transactions. Ensure ordered CID

CPU times: user 9.96 s, sys: 2.41 s, total: 12.4 s
Wall time: 13.3 s


,cid,dt,tm,amt,mcc,ttc,tid
0,0,1,00:00:00,-1305,56,4,NaN
1,0,1,00:00:00,-4821,56,4,NaN
2,0,2,22:48:17,-56156,22,37,NaN
3,0,3,00:00:00,-3708,56,4,NaN
...,...,...,...,...,...,...,...
3749574,8399,450,00:00:00,-43831,56,4,PS026577
3749575,8399,456,17:21:07,89833,123,128,888917
3749576,8399,456,18:59:26,247062,124,108,738051
3749577,8399,456,19:01:21,-336874,22,25,738051


In [14]:
df_ = gender.drop('gender', axis=1); df_.T  # a list of ordered client IDs (as index)

cid,0,1,2,3,4,...,8395,8396,8397,8398,8399


In [15]:
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 20, 'display.width', 1000, 'display.max_rows', 10)
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 120 sec. Good luck!


# **My Code**

In [140]:
mcc_by_cid = dfTrx[['cid', 'mcc']].drop_duplicates().astype('str').groupby('cid').agg({'mcc': ' '.join}).sort_values('cid')
sum_amt_by_cid = dfTrx[['cid', 'amt']].groupby('cid').agg('sum')

In [141]:
large_count = dfTrx[['cid', 'mcc']].drop_duplicates().groupby('mcc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_sum = dfTrx[['amt', 'mcc']].groupby('mcc').agg('sum').sort_values(by = 'amt')
most_popular_mcc = set(large_count[large_count.cid > 2000].index.to_list()) | set(large_sum[large_sum.amt < -1000000000].index.to_list()) | set([123, 125, 130])
len(most_popular_mcc)

31

In [142]:
#@title Начало создания df по cid

df = df_.copy(deep=True)
for mcc_index in most_popular_mcc:
    sum_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
    df = df.join(sum_amt_by_cid_and_mcc.amt, on = 'cid', how='left').rename(columns = {'amt': str(mcc_index)})
df = df.fillna(0).astype('int')

# **Обучение**

In [166]:
%%time
trainX, testX = df.join(trainY.drop('gender', axis=1), how='inner'), df.join(testY.drop('gender', axis=1), how='inner') # split observations into train/test
trainX0, testX0, trainY0, testY0 = train_test_split(trainX, trainY, test_size=0.3, random_state=0) # split train observation into local train/test sets (to score performance)
model = LR(random_state=0, n_jobs=-1, max_iter=1000)                             # always seed your experiments for reproducibility

print('Local AUC ROC', roc_auc_score(testY0.gender, model.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]))       # fit the model and show test error

Local AUC ROC 0.5
CPU times: user 52.6 ms, sys: 6.25 ms, total: 58.8 ms
Wall time: 2.3 s
